# Spurs Data Analysis

### I loaded the necessary packages

In [307]:
import pandas as pd
import numpy as np
import csv
import os

### I imported all the csvs from the <b>"raw_season_csvs"</b> folder and combined them into one dataframe
It appears to all be correct (each season should have 380 total games, so one game per row results in 380 rows per file).

<br><i>The data comes from [football-data.co.uk](https://www.football-data.co.uk/englandm.php)</i>
<br><i>The below cell was written with the help of Claude Code</i>

In [308]:
csv_dir = "/Users/hgorledeenn/Desktop/CJS/0126data_studio/project-1-spurs/raw_season_csvs"

csv_files = sorted([f for f in os.listdir(csv_dir) if f.endswith('.csv')])
all_rows = []
all_columns = set()

for file in csv_files:
    filepath = os.path.join(csv_dir, file)
    row_count = 0
    with open(filepath, 'r', encoding='latin-1') as f:
        reader = csv.DictReader(f)
        for row in reader:
            row['season'] = file.split('.')[0].split('-')[1]
            all_rows.append(row)
            all_columns.update(row.keys())
            row_count += 1
    print(f"Loaded {file}: {row_count} rows")

combined_df = pd.DataFrame(all_rows)

print(f"\nCombined dataframe shape: {combined_df.shape}")
print(f"Number of columns: {len(combined_df.columns)}")


Loaded 10-0203.csv: 380 rows
Loaded 11-0304.csv: 380 rows
Loaded 12-0405.csv: 380 rows
Loaded 13-0506.csv: 380 rows
Loaded 14-0607.csv: 380 rows
Loaded 15-0708.csv: 380 rows
Loaded 16-0809.csv: 380 rows
Loaded 17-0910.csv: 380 rows
Loaded 18-1011.csv: 380 rows
Loaded 19-1112.csv: 380 rows
Loaded 20-1213.csv: 380 rows
Loaded 21-1314.csv: 380 rows
Loaded 22-1415.csv: 381 rows
Loaded 23-1516.csv: 380 rows
Loaded 24-1617.csv: 380 rows
Loaded 25-1718.csv: 380 rows
Loaded 26-1819.csv: 380 rows
Loaded 27-1920.csv: 380 rows
Loaded 28-2021.csv: 380 rows
Loaded 29-2122.csv: 380 rows
Loaded 30-2223.csv: 380 rows
Loaded 31-2324.csv: 380 rows
Loaded 32-2425.csv: 380 rows
Loaded 33-2526.csv: 260 rows
Loaded 8-0001.csv: 380 rows
Loaded 9-0102.csv: 380 rows

Combined dataframe shape: (9761, 218)
Number of columns: 218


### I selected only the columns I need for my analysis and saved the pared down dataframe as a csv

In [309]:
all_games = combined_df[['season','Date', 'HomeTeam','AwayTeam','FTHG','FTAG','FTR','HTHG','HTAG','HTR','HS','AS','HST','AST']]

In [310]:
all_games.to_csv('all_games.csv', index=False)

In [311]:
all_games.head()

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST
0,0203,17/08/2002,Blackburn,Sunderland,0,0,D,0,0,D,15,7,5,3
1,0203,17/08/2002,Charlton,Chelsea,2,3,A,2,1,H,5,21,5,12
2,0203,17/08/2002,Everton,Tottenham,2,2,D,1,0,H,13,10,9,5
3,0203,17/08/2002,Fulham,Bolton,4,1,H,3,1,H,13,3,6,1
4,0203,17/08/2002,Leeds,Man City,3,0,H,2,0,H,13,18,8,10


### Filter to only games where Tottenham is the home or away team, reset the index, standardize the date column

In [312]:
only_tot = all_games[(all_games['HomeTeam'] == 'Tottenham') | (all_games['AwayTeam'] == 'Tottenham')]
only_tot = only_tot.reset_index(drop=True)
only_tot['Date'] = pd.to_datetime(only_tot['Date'], dayfirst=True, format='mixed')
only_tot.head()

,season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,HS,AS,HST,AST
0,0203,2002-08-17,Everton,Tottenham,2,2,D,1,0,H,13,10,9,5
1,0203,2002-08-24,Tottenham,Aston Villa,1,0,H,1,0,H,8,18,3,8
2,0203,2002-08-27,Charlton,Tottenham,0,1,A,0,1,A,10,7,5,1
3,0203,2002-08-31,Tottenham,Southampton,2,1,H,1,1,D,18,15,8,8
4,0203,2002-09-11,Fulham,Tottenham,3,2,H,0,2,A,13,7,8,4


### Make a blank dataframe for my tottenham-centric stats

In [313]:
tottenham = pd.DataFrame()

### Defining a few functions for use below

In [314]:
def ft_results(row):
    if row['TFTG'] > row['OTFTG']:
        return 'W'
    elif row['TFTG'] < row['OTFTG']:
        return 'L'
    else:
        return 'D'
    
def ht_results(row):
    if row['THTG'] > row['OTHTG']:
        return 'W'
    elif row['THTG'] < row['OTHTG']:
        return 'L'
    else:
        return 'D'

### Below, I populate my new dataframe with the same stats as in ```only_tot```, just from Tottenham's perspective (mostly just using np.where to determine whether to pull in the home team stat or the away team stat)

#### Data Diary for my new columns:
<b>Season</b>: Season of the game (in the format 0203, where the season spanned 2002 and 2003)
<br><b>Date</b>: Date of the game
<br><b>OT</b>: The team Tottenham played
<br><b>TL</b>: The location of the game, from Tottenham's perspective (H = Home, A = Away)
<br><b>TFTG</b>: Tottenham Full-Time Goals
<br><b>OTFTG</b>: Other Team Full-Time Goals
<br><b>FTR</b>: Full Time Result (W if Tottenham win, L if Tottenham loss, D if draw)
<br><b>THTG</b>: Tottenham Half-Time Goals
<br><b>OTHTG</b>: Other Team Half-Time Goals
<br><b>HTR</b>: Half Time Result (W if Tottenham winning, L if Tottenham losing, D if tied)
<br><b>TS</b>: Total shots by Tottenham
<br><b>OTS</b>: Total shots by the Other Team
<br><b>TST</b>: Tottenham shots on target
<br><b>OTST</b>: Other Team shots on target
<br><b>FTGD</b>: Full-Time goal differential (+ if Tottenham won, - if Tottenham lost, 0 if draw)
<br><b>HTGD</b>: Half-Time goal differential (+ if Tottenham won, - if Tottenham lost, 0 if draw)
<br><b>OTST</b>: The last name of Tottenham's head coach for that game
<br><b>Coach</b>: The last name of Tottenham's head coach for that game
<br><b>Coach</b>: The last name of Tottenham's head coach for that game
<br><b>Coach</b>: The last name of Tottenham's head coach for that game
<br><b>Coach</b>: The last name of Tottenham's head coach for that game
<br><b>Coach</b>: The last name of Tottenham's head coach for that game
<br><b>Coach</b>: The last name of Tottenham's head coach for that game

In [315]:
tottenham['Season'] = only_tot['season']
tottenham['Date'] = only_tot['Date']
tottenham['OT'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['AwayTeam'], only_tot['HomeTeam'])
tottenham['TL'] = np.where(only_tot['HomeTeam'] == 'Tottenham', "H", "A")
tottenham['TFTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['FTHG'], only_tot['FTAG'])
tottenham['OTFTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['FTAG'], only_tot['FTHG'])
tottenham['FTR'] = tottenham.apply(ft_results, axis=1)
tottenham['THTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HTHG'], only_tot['HTAG'])
tottenham['OTHTG'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HTAG'], only_tot['HTHG'])
tottenham['HTR'] = tottenham.apply(ht_results, axis=1)
tottenham['TS'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HS'], only_tot['AS'])
tottenham['OTS'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['AS'], only_tot['HS'])
tottenham['TST'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['HST'], only_tot['AST'])
tottenham['OTST'] = np.where(only_tot['HomeTeam'] == 'Tottenham', only_tot['AST'], only_tot['HST'])


In [316]:
numeric_columns = ['TFTG', 'OTFTG', 'THTG', 'OTHTG', 'TS', 'OTS', 'TST', 'OTST']

for col in numeric_columns:
    tottenham[col] = pd.to_numeric(tottenham[col], errors='coerce')

tottenham.dtypes

Season            object
Date      datetime64[ns]
OT                object
TL                object
TFTG               int64
OTFTG              int64
FTR               object
THTG               int64
OTHTG              int64
HTR               object
TS                 int64
OTS                int64
TST                int64
OTST               int64
dtype: object

In [317]:
tottenham['FTGD'] = tottenham['TFTG'] - tottenham['OTFTG']
tottenham['HTGD'] = tottenham['THTG'] - tottenham['OTHTG']


## Next, I created a dataframe with each Tottenham coach and their start and end dates in the time period my data covers

In [318]:
import pandas as pd

coaches = [
    {"coach":"George Graham","start":"1998-10-01","end":"2001-03-16"},
    {"coach":"Interim Manager","start":"2001-03-17","end":"2001-04-01"},
    {"coach":"Glen Hoddle","start":"2001-04-02","end":"2003-09-21"},
    {"coach":"Interim Manager","start":"2003-09-22","end":"2004-06-03"},
    {"coach":"Jacques Santini","start":"2004-06-04","end":"2004-11-05"},
    {"coach":"Martin Jol","start":"2004-11-06","end":"2007-10-26"},
    {"coach":"Interim Manager","start":"2007-10-27","end":"2007-10-29"},
    {"coach":"Juande Ramos","start":"2007-10-30","end":"2008-10-25"},
    {"coach":"Harry Redknapp","start":"2008-10-26","end":"2012-06-15"},
    {"coach":"André Villas-Boas","start":"2012-06-16","end":"2013-12-16"},
    {"coach":"Tim Sherwood","start":"2013-12-17","end":"2014-05-13"},
    {"coach":"Mauricio Pochettino","start":"2014-05-14","end":"2019-11-19"},
    {"coach":"José Mourinho","start":"2019-11-20","end":"2021-04-19"},
    {"coach":"Interim Manager","start":"2021-04-20","end":"2021-06-29"},
    {"coach":"Nuno Espírito Santo","start":"2021-06-30","end":"2021-11-01"},
    {"coach":"Antonio Conte","start":"2021-11-02","end":"2023-03-26"},
    {"coach":"Interim Manager","start":"2023-03-27","end":"2023-06-05"},
    {"coach":"Ange Postecoglou","start":"2023-06-06","end":"2025-06-06"},
    {"coach":"Thomas Frank","start":"2025-06-07","end":"2026-02-11"},
    {"coach":"Igor Tudor","start":"2026-02-12","end":"2026-03-01"}
]

coach_df = pd.DataFrame(coaches)
coach_df["start"] = pd.to_datetime(coach_df["start"])
coach_df["end"]   = pd.to_datetime(coach_df["end"])


In [319]:
tottenham = tottenham.sort_values("Date")
coach_df = coach_df.sort_values("start")

tottenham = pd.merge_asof(tottenham, coach_df, left_on="Date", right_on="start", direction="backward")
tottenham = tottenham[tottenham["Date"] <= tottenham["end"]]

tottenham

,Season,Date,OT,TL,TFTG,OTFTG,FTR,THTG,OTHTG,HTR,TS,OTS,TST,OTST,FTGD,HTGD,coach,start,end
0,0001,2000-08-19,Ipswich,H,3,1,W,2,1,W,20,15,6,5,2,1,George Graham,1998-10-01,2001-03-16
1,0001,2000-08-22,Middlesbrough,A,1,1,D,1,0,W,11,12,4,6,0,1,George Graham,1998-10-01,2001-03-16
2,0001,2000-08-26,Newcastle,A,0,2,L,0,1,L,10,15,2,6,-2,-1,George Graham,1998-10-01,2001-03-16
3,0001,2000-09-05,Everton,H,3,2,W,1,2,L,11,11,7,6,1,-1,George Graham,1998-10-01,2001-03-16
4,0001,2000-09-11,West Ham,H,1,0,W,0,0,D,15,11,7,5,1,0,George Graham,1998-10-01,2001-03-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,2526,2026-01-17,West Ham,H,1,2,L,0,1,L,21,17,6,4,-1,-1,Thomas Frank,2025-06-07,2026-02-11
972,2526,2026-01-24,Burnley,A,2,2,D,1,1,D,18,9,11,4,0,0,Thomas Frank,2025-06-07,2026-02-11
973,2526,2026-02-01,Man City,H,2,2,D,0,2,L,12,15,6,3,0,-2,Thomas Frank,2025-06-07,2026-02-11
974,2526,2026-02-07,Man United,A,0,2,L,0,1,L,7,23,1,10,-2,-1,Thomas Frank,2025-06-07,2026-02-11


In [320]:
tottenham = tottenham.drop(columns=['start', 'end'])

In [321]:
tottenham.to_csv('tottenham_centric.csv', index=False, encoding="utf-8-sig")
tottenham.head()

,Season,Date,OT,TL,TFTG,OTFTG,FTR,THTG,OTHTG,HTR,TS,OTS,TST,OTST,FTGD,HTGD,coach
0,0001,2000-08-19,Ipswich,H,3,1,W,2,1,W,20,15,6,5,2,1,George Graham
1,0001,2000-08-22,Middlesbrough,A,1,1,D,1,0,W,11,12,4,6,0,1,George Graham
2,0001,2000-08-26,Newcastle,A,0,2,L,0,1,L,10,15,2,6,-2,-1,George Graham
3,0001,2000-09-05,Everton,H,3,2,W,1,2,L,11,11,7,6,1,-1,George Graham
4,0001,2000-09-11,West Ham,H,1,0,W,0,0,D,15,11,7,5,1,0,George Graham


### I calcualted some rolling averages for certain stats

In [322]:
tottenham['FTGDRA'] = tottenham['FTGD'].rolling(window=5).mean()
tottenham['HTGDRA'] = tottenham['HTGD'].rolling(window=5).mean()
tottenham['TFTGRA'] = tottenham['TFTG'].rolling(window=5).mean()
tottenham['OTFTGRA'] = tottenham['OTFTG'].rolling(window=5).mean()
tottenham['TSTRA'] = tottenham['TST'].rolling(window=5).mean()
tottenham['OTSTRA'] = tottenham['OTST'].rolling(window=5).mean()

In [323]:
tottenham.head(15)

,Season,Date,OT,TL,TFTG,OTFTG,FTR,THTG,OTHTG,HTR,...,OTST,FTGD,HTGD,coach,FTGDRA,HTGDRA,TFTGRA,OTFTGRA,TSTRA,OTSTRA
0,0001,2000-08-19,Ipswich,H,3,1,W,2,1,W,...,5,2,1,George Graham,NaN,NaN,NaN,NaN,NaN,NaN
1,0001,2000-08-22,Middlesbrough,A,1,1,D,1,0,W,...,6,0,1,George Graham,NaN,NaN,NaN,NaN,NaN,NaN
2,0001,2000-08-26,Newcastle,A,0,2,L,0,1,L,...,6,-2,-1,George Graham,NaN,NaN,NaN,NaN,NaN,NaN
3,0001,2000-09-05,Everton,H,3,2,W,1,2,L,...,6,1,-1,George Graham,NaN,NaN,NaN,NaN,NaN,NaN
4,0001,2000-09-11,West Ham,H,1,0,W,0,0,D,...,5,1,0,George Graham,0.4,0.0,1.6,1.2,5.2,5.6
5,0001,2000-09-16,Charlton,A,0,1,L,0,1,L,...,5,-1,-1,George Graham,-0.2,-0.4,1.0,1.2,5.0,5.6
6,0001,2000-09-23,Man City,H,0,0,D,0,0,D,...,2,0,0,George Graham,-0.2,-0.6,0.8,1.0,5.2,4.8
7,0001,2000-09-30,Leeds,A,3,4,L,1,0,W,...,7,-1,1,George Graham,0.0,-0.2,1.4,1.4,5.8,5.0
8,0001,2000-10-14,Coventry,A,1,2,L,0,2,L,...,5,-1,-2,George Graham,-0.4,-0.4,1.0,1.4,5.4,4.8
9,0001,2000-10-21,Derby,H,3,1,W,2,1,W,...,7,2,1,George Graham,-0.2,-0.2,1.4,1.6,5.6,5.2


In [324]:
tottenham.to_csv('tottenham_centric.csv', index=False, encoding="utf-8-sig")